# Librerías

In [1]:
# cálculos
import numpy as np
import pandas as pd
import math

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# modelos
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# metricas
import sklearn.metrics as metrics

# hiperparametrizado
from sklearn.model_selection import GridSearchCV

# quitar alertas
import warnings
warnings.filterwarnings("ignore")



In [3]:
# Ruta al archivo 
ruta_archivo = "C:/Users/Herre/Desktop/NUCLIO/Entrega_TFM/modelo_investment.csv"
# Leer el archivo 
df_modelo = pd.read_csv(ruta_archivo)

In [5]:
# En CatBoost no hace falta estandarizar

In [4]:
# 1. Dividir los datos en variables predictoras y target
# Asumamos que la columna 'category_investment' es la que queremos predecir
X = df_modelo.drop('category_investment', axis=1)  # Eliminar la columna target del conjunto de datos
y = df_modelo['category_investment']  # Definir la columna objetivo

# 2. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Toreno de modelos

In [5]:
f1s = pd.DataFrame(columns=['modelo', 'f1_score'])
f1s

,modelo,f1_score


In [6]:
# añadimos en una lista los modelos que queremos poner a competir

modelos = []
modelos.append(('SVC', SVC()))
modelos.append(('LogisticRegression', LogisticRegression()))
modelos.append(('RidgeClassifier', RidgeClassifier()))
modelos.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
modelos.append(('RandomForestClassifier', RandomForestClassifier()))
modelos.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
modelos.append(('XGBClassifier', XGBClassifier()))
modelos.append(('CatBoostClassifier', CatBoostClassifier(verbose=False)))

In [7]:
for nombre, modelo in modelos:
  # entrenamiento
  modelo.fit(X_train, y_train)

  # predicción del test
  y_pred = modelo.predict(X_test)

  # evalua la metrica
  f1_score = metrics.f1_score(y_test, y_pred, average='macro')
  #el average='macro' se usa cuando estamos haciendo modelos multiclases. Sino lo pones el modelo peta.

  # añade la métrica al dataframe de métricas
  metrica = pd.DataFrame({'modelo':[nombre], 'f1_score':[f1_score]})

  print(nombre, np.round(f1_score,5))

  f1s = pd.concat([f1s,metrica], ignore_index=True)

SVC 0.51548
LogisticRegression 0.70655
RidgeClassifier 0.83695
DecisionTreeClassifier 0.79201
RandomForestClassifier 0.85843
GradientBoostingClassifier 0.86368
XGBClassifier 0.85834
CatBoostClassifier 0.86586


In [8]:
f1s.sort_values('f1_score', ascending= False)

,modelo,f1_score
7,CatBoostClassifier,0.865864
5,GradientBoostingClassifier,0.863677
4,RandomForestClassifier,0.858429
6,XGBClassifier,0.858339
2,RidgeClassifier,0.836951
3,DecisionTreeClassifier,0.792010
1,LogisticRegression,0.706549
0,SVC,0.515478


# Modelo

In [9]:
# Definir el modelo de CatBoost
# No necesitamos especificar las características categóricas manualmente
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

0:	learn: 0.5825083	total: 7.66ms	remaining: 7.66s
100:	learn: 0.3024858	total: 619ms	remaining: 5.5s
200:	learn: 0.2898113	total: 1.25s	remaining: 4.96s
300:	learn: 0.2794708	total: 1.86s	remaining: 4.33s
400:	learn: 0.2709426	total: 2.48s	remaining: 3.7s
500:	learn: 0.2621439	total: 3.09s	remaining: 3.07s
600:	learn: 0.2549934	total: 3.69s	remaining: 2.45s
700:	learn: 0.2476921	total: 4.31s	remaining: 1.84s
800:	learn: 0.2413005	total: 4.9s	remaining: 1.22s
900:	learn: 0.2350303	total: 5.5s	remaining: 605ms
999:	learn: 0.2287599	total: 6.1s	remaining: 0us
Accuracy: 0.8645
ROC AUC: 0.9157


In [10]:
# Nos aseguramos de que la columna 'pk_cid' esté en X_test

# Crear un nuevo DataFrame con pk_cid, las predicciones y las probabilidades
df_resultado = X_test[['pk_cid']].copy()  # Copiamos la columna pk_cid de X_test
df_resultado['prediccion'] = y_pred  # Agregamos la columna de predicciones
df_resultado['probabilidad_compra'] = y_pred_proba  # Agregamos la columna de probabilidades

# Mostrar el nuevo DataFrame directamente en pantalla
print(df_resultado.head())  # Muestra las primeras filas del DataFrame

        pk_cid  prediccion  probabilidad_compra
4969   1413059           1             0.852624
36503  1325106           1             0.703597
21867  1140945           0             0.015178
36058  1203768           0             0.000128
14458  1214866           0             0.002784


In [25]:
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión: {accuracy * 100:.2f}%")

Precisión: 86.52%


In [15]:
# Revisar la distribución de la variable objetivo
df_resultado['prediccion'].value_counts(normalize=True)

prediccion
1    0.5782
0    0.4218
Name: proportion, dtype: float64

In [16]:
df_resultado["probabilidad_compra"].max()

0.9994243724264922

Necesitamos a los clientes con una buena probabilidad de compra, es por eso que escogemos a aquellos con una probabilidad superior al 70%.

In [11]:
# Filtrar filas donde la probabilidad de compra sea mayor al 60%
df_filtrado = df_resultado[df_resultado['probabilidad_compra'] > 0.70]

In [23]:
df_filtrado

,pk_cid,prediccion,probabilidad_compra
4969,1413059,1,0.870315
50255,1264917,1,0.788055
6361,1174487,1,0.853140
17255,1239811,1,0.880158
8939,1392989,1,0.704475
...,...,...,...
11026,1521133,1,0.795437
24360,1235346,1,0.762751
29653,1519030,1,0.784230
16188,1173595,1,0.844871


Exportamos la prediccion porque la necesitaremos más adelante para conocer los ingresos que tendrá la empresa. Nos quedamos con todos los clientes con una probabilidad de comrpa superiro al 70% porque son solo 4991.

In [12]:
# Guardar el DataFrame en un archivo CSV para revisarlo luego
df_filtrado.to_csv('investment_resultado_predicciones.csv', index=False)